# Library Import


In [3]:
import wave
import numpy as np
import matplotlib.pyplot as plt
from Crypto.Hash import SHA3_256
from Crypto.Cipher import ChaCha20_Poly1305
from Crypto.Random import get_random_bytes

# E91


# SHA-3 Digest


In [65]:
aliceKey = get_random_bytes(500)
print("Alice's Key: ", aliceKey)
hashObject = SHA3_256.new()
hashObject.update(aliceKey)
key = hashObject.digest()
print("Hash of Alice's key: ", key)

Alice's Key:  b'=x\x12,\x89\xf3]\xfat\xc5\x12**\x9f\x11\xf1q~G\x9c}\xa9\x0bw\xb5\xc8\xb28\xc4B\xa6\x9fn\xf6\x01#\xe0\x0c88+g\x85EI!%\xe8\xd3\xf0E\x81\xec\xc3\nI;#\xca\xdd\x1d\xda\xc6\xf7\xcb\xc8\n+\xe4L|33\xde"cNu\xae\xaa\xb8\x9c\xc6/M+\x12\xc4\x1b\x91\xa7O\xd7\xd7H\xe0|\xcc:3?\x8a\xd1$B,~\x87m\xab4\xc7\x19`\xa2\xf7\xf1\xef5ag\'\x92C\xd26\xf5\xbc\x05i\xe4tC_^x\xb0\x10\xfa>-C5Y\x0c\xb8\x98+\x1d.|\xbcx\x14{[\x1a\x07\xb7\xec\x03\xcf\xcdr\xfc\x8a]\\Gq\x93\x94\x89\xc3\x90\xd3\x15\xf7\x9b\xcf/\x10d\xc10B\xd2\x16\xd6\x86i\xf4(\xa2B\xee\x91\x1a1RI\\\x02\xdf\xd1!\xf7\x04\xba\xff\xd7\xc9\x8e\xc1[\xf0w\x94]j\xfant\xac\xe8_7,y\xc1\xa5\x96\xe4\xd3U\xbeF\xff\x8a1O\xcbS\xd7\x93\x99\xa5"P\x9c\x9f\x88)c!\xd1\xc6\xdd\xbdI\xc4\xee\x11)\xaeZ\xbcK\x14\x19\xdb\xd1\xe9\xa14"\xa9h\xcf<\xf8[\x18\xe7\xd6Obx\x80\x1f@\xa2\xb6t\x1c\xe0\x19\x96+\x10\x13\\\xcc\x87s\xd2Q=\xd1U\xefG\xfb2b_\x03\x1d\xbd\xf74\x98#!O\x7f6\xe0\xea^\xa8D\x02t\xff&!tPy)\xa3jV\x91\xfce\xda\xb4\xe1\xfbh\x00\xdf\x02\x85\xf9\xcc\xbc\x11\xe9\xc0\

# Steganography


## Function Definition


### Embedding


#### Audio Clipping Function


In [66]:
def clip_audio(audio_data, min_value=-32768, max_value=32767):
    return np.clip(audio_data, min_value, max_value)

#### Audio Embedding Function


In [67]:
def embed_audio(carrier_path, secret_path, output_path, num_bits=8):
    # Open the carrier and secret audio files
    carrier = wave.open(carrier_path, 'rb')
    secret = wave.open(secret_path, 'rb')
    # Ensure compatibility between carrier and secret audio
    if carrier.getnchannels() != secret.getnchannels() or carrier.getsampwidth() != secret.getsampwidth():
        raise ValueError(
            "Carrier and secret audio must have the same number of channels and sample width.")
    # Read audio data
    carrier_frames = np.frombuffer(carrier.readframes(
        carrier.getnframes()), dtype=np.int16)
    secret_frames = np.frombuffer(secret.readframes(
        secret.getnframes()), dtype=np.int16)
    # Ensure carrier has enough capacity to store secret
    max_secret_length = len(carrier_frames) // (16 // num_bits)
    if len(secret_frames) > max_secret_length:
        raise ValueError(f"Carrier audio cannot hold the secret audio. Max capacity: {
                         max_secret_length}, got: {len(secret_frames)}.")
    # Resize the secret frames to match embedding capacity
    padded_secret_frames = np.zeros(max_secret_length, dtype=np.int16)
    padded_secret_frames[:len(secret_frames)] = secret_frames
    # Embed the secret audio into the carrier
    embedded_frames = np.copy(carrier_frames)
    mask = (1 << num_bits) - 1  # Mask to isolate the LSBs
    embedded_frames &= ~mask  # Clear the least significant bits
    embedded_frames[:len(padded_secret_frames)] += (padded_secret_frames >>
                                                    # Embed secret bits
                                                    (16 - num_bits)) & mask
    # Clip audio values to valid range
    embedded_frames = clip_audio(embedded_frames)
    # Write the embedded audio to a new file
    embedded_audio = wave.open(output_path, 'wb')
    embedded_audio.setparams(carrier.getparams())
    embedded_audio.writeframes(embedded_frames.astype(np.int16).tobytes())
    # Close files
    carrier.close()
    secret.close()
    embedded_audio.close()
    print(f"Embedding complete. Saved to {output_path}")

### Extraction


In [68]:
def extract_audio(embedded_path, secret_output_path, secret_length, num_bits=8):
    # Open the embedded audio file
    embedded = wave.open(embedded_path, 'rb')
    # Read audio data
    embedded_frames = np.frombuffer(embedded.readframes(
        embedded.getnframes()), dtype=np.int16)
    # Extract the secret audio
    mask = (1 << num_bits) - 1  # Mask to isolate the embedded bits
    extracted_bits = (embedded_frames & mask) << (16 - num_bits)
    secret_frames = extracted_bits[:secret_length*2]
    # Apply a low-pass filter to reduce noise
    secret_frames = np.convolve(secret_frames, np.ones(
        5)/5, mode='same')  # Simple smoothing (moving average)
    # Clip audio values to valid range
    secret_frames = clip_audio(secret_frames)
    # Write the secret audio to a new file
    secret_audio = wave.open(secret_output_path, 'wb')
    secret_audio.setparams(embedded.getparams())
    secret_audio.setnframes(secret_length)
    secret_audio.writeframes(secret_frames.astype(np.int16).tobytes())
    # Close files
    embedded.close()
    secret_audio.close()
    print(f"Extraction complete. Secret audio saved to {secret_output_path}")

## Implementation


### Input


In [69]:
# Carrier audio
carrier_audio = "cover.wav"
secret_audio = "secret2.wav"  # Secret audio
embedded_audio = "embedded2.wav"  # Output for the embedded audio

### Embed


In [70]:
embed_audio(carrier_audio, secret_audio, embedded_audio, num_bits=8)

Embedding complete. Saved to embedded2.wav


# Encryption, Decryption, MAC Generation and Authentication


## Function Definitions


Encryption using the SHA3-256 digest of the final Ekert key.


In [71]:
def encryptAudio(key, inputFile, outputFile, outputAudio):
    with open(inputFile, 'rb') as inputFile:
        audioData = inputFile.read()
    header = b"Stego Audio"
    cipher = ChaCha20_Poly1305.new(key=key)
    cipher.update(header)
    nonce = cipher.nonce
    print("Nonce: ", nonce.hex())
    ciphertext, tag = cipher.encrypt_and_digest(audioData)
    print("Tag: ", tag.hex())
    with open(outputAudio, 'wb') as outputAudio:
        outputAudio.write(ciphertext)
    with open(outputFile, 'wb') as outputFile:
        outputFile.write(nonce+ciphertext+tag)

Applying random bit flip to encrypted audio in order to simulate middle-man attack


In [72]:
def addNoise(inputFile, noisyOutputFile):
    with open(inputFile, 'rb') as inputFile:
        noisyCiphertextBytes = bytearray(inputFile.read())
        for x in range(100000):
            indexToModify = np.random.randint(0, len(noisyCiphertextBytes)-1)
            noisyCiphertextBytes[indexToModify] ^= np.random.randint(
                0x00, 0xFF)
        with open(noisyOutputFile, 'wb') as noisyOutputFile:
            noisyOutputFile.write(noisyCiphertextBytes)

Decryption


In [73]:
def decryptAudio(key, inputFile, outputFile):
    with open(inputFile, 'rb') as inputFile:
        data = inputFile.read()
        nonce = data[:12]
        ciphertext = data[12:-16]
        tag = data[-16:]
    header = b"Stego Audio"
    cipher = ChaCha20_Poly1305.new(key=key, nonce=nonce)
    cipher.update(header)
    try:
        decryptedAudio = cipher.decrypt_and_verify(ciphertext, tag)
        with open(outputFile, 'wb') as outputFile:
            outputFile.write(decryptedAudio)
            print("Nonce: ", nonce.hex())
            print("Tag: ", tag.hex())
            print("Decryption and verification successful!")
    except ValueError:
        print("Verification failed!")

## Implementation


In [74]:
inputAudio = "embedded2.wav"
outputAudio = "encrypted2.wav"
output = "encrypted2.bin"
decryptedAudio = "decrypted2.wav"

In [75]:
encryptAudio(key, inputAudio, output, outputAudio)

Nonce:  7eb7d80f21b86f3a7fa7e16b
Tag:  9e6a1b409637aaa3c8c050d5a2ee8934


In [76]:
addNoise("encrypted2.bin", "noisy2.bin")
noisyOutput = "noisy2.bin"

In [77]:
decryptAudio(key, output, decryptedAudio)

Nonce:  7eb7d80f21b86f3a7fa7e16b
Tag:  9e6a1b409637aaa3c8c050d5a2ee8934
Decryption and verification successful!


In [78]:
decryptAudio(key, noisyOutput, "outfile.bin")

Verification failed!


# Extraction


In [79]:
extracted_audio = "retrieved_audio2.wav"
# extract_audio('embedded.wav', 'output.wav', 8)

secret_audio_length = wave.open(
    secret_audio, 'rb').getnframes()  # Length of the secret audio
extract_audio(embedded_audio, extracted_audio, secret_audio_length)

Extraction complete. Secret audio saved to retrieved_audio2.wav


# Analysis


Add embedded.wav and encrypted.bin as argument

In [4]:
def createByteArray(embeddedFile, encryptedBinFile):
    with open(embeddedFile, 'rb') as f:
        embeddedAudio = bytearray(f.read())
    with open(encryptedBinFile, 'rb') as f:
        data = f.read()
        ciphertext = data[12:-16]
        encryptedAudio = bytearray(ciphertext)
    byteArray = [embeddedAudio, encryptedAudio]
    return byteArray

## Entropy Calculation and Comparison

In [1]:
def calculateEntropy(embeddedFile, encryptedBinFile):
    embeddedAudio, encryptedAudio = createByteArray(
        embeddedFile, encryptedBinFile)
    audioFiles = [embeddedAudio, encryptedAudio]
    for x in audioFiles:
        hist, _ = np.histogram(x, bins=256, range=(0, 256))
        # Calculate probability distribution
        probabilities = hist / len(x)
        # Remove zeros from probabilities (to avoid log2(0) errors)
        probabilities = probabilities[probabilities > 0]
        # Calculate entropy
        entropy = -np.sum(probabilities * np.log2(probabilities))
        if x == embeddedAudio:
            print("Entropy of Original Audio: ", entropy)
        else:
            print("Entropy of Encrypted Audio: ", entropy)

In [5]:
calculateEntropy('embedded1.wav', 'encrypted1.bin')

Entropy of Original Audio:  5.343236996344665
Entropy of Encrypted Audio:  7.99999700154831


## Correlation Calculation

In [9]:
def calculateCorrelation(embeddedFile, encryptedBinFile):
    embeddedAudio, encryptedAudio = createByteArray(
        embeddedFile, encryptedBinFile)
    correlation = np.corrcoef(embeddedAudio, encryptedAudio)[0, 1]
    print("Correlation Coefficient:", correlation)

In [11]:
calculateCorrelation('embedded1.wav', 'encrypted1.bin')

Correlation Coefficient: -0.0001969823910425479


## UACI Calculation

In [2]:
def calculateUACI(embeddedFile, encryptedBinFile):
    embeddedAudio, encryptedAudio = createByteArray(
        embeddedFile, encryptedBinFile)
    maxVal = np.max(np.abs(embeddedAudio))
    diff = np.abs(np.subtract(embeddedAudio, encryptedAudio))
    sumDiff = np.sum(diff)/len(diff)
    uaci = sumDiff/maxVal*100
    print("UACI: ", uaci)

In [5]:
calculateUACI('embedded1.wav', 'encrypted1.bin')

UACI:  49.998294871449374


## NSCR Calculation

In [6]:
def calculateNSCR(embeddedFile, encryptedBinFile):
    embeddedAudio, encryptedAudio = createByteArray(
        embeddedFile, encryptedBinFile)
    nscr = 0
    for i in range(len(embeddedAudio)):
        if embeddedAudio[i] != encryptedAudio[i]:
            nscr += 1
    nscr = nscr/len(embeddedAudio)*100
    print("NSCR: ", nscr)

In [7]:
calculateNSCR('embedded1.wav', 'encrypted1.bin')

NSCR:  99.60953638738931
